In [1]:
import pandas as pd
import numpy as np
import cv2
import os
import re

from PIL import Image

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

import torch
import torchvision
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SequentialSampler
import matplotlib.pyplot as plt


In [2]:
from detecto import utils
from sklearn.model_selection import train_test_split

utils.xml_to_csv('train_data/annotations', 'train_data/train.csv')
train_df = pd.read_csv('train_data/train.csv')

train_df, val_df = train_test_split(train_df, test_size=0.1, random_state=42)
print(train_df.shape)
print(val_df.shape)

(3664, 8)
(408, 8)


In [3]:
class Dataset(Dataset):

    def __init__(self, dataframe, image_dir, transforms=None):
        super().__init__()
        self.image_ids = dataframe['filename'].unique()
        self.df = dataframe
        self.image_dir = image_dir
        self.transforms = transforms

    def __getitem__(self, index: int):

        image_id = self.image_ids[index]
        records = self.df[self.df['filename'] == image_id]
        image = cv2.imread(f'{self.image_dir}/{image_id}', cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0

        boxes = records[['xmin', 'ymin', 'xmax', 'ymax']].values
        
        
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        area = torch.as_tensor(area, dtype=torch.float32)
#         labels = torch.ones((records.shape[0],), dtype=torch.int64)
        labels=[]
        for i in range(records.shape[0]):
            label = records.iloc[i,3]
            if(label=='with_mask'):
                labels.append(2)
            else:
                labels.append(1)
        labels = torch.as_tensor(labels)
        # suppose all instances are not crowd
        iscrowd = torch.zeros((records.shape[0],), dtype=torch.int64)
        
        target = {}
        target['boxes'] = boxes
        target['labels'] = labels
        # target['masks'] = None
        target['image_id'] = torch.tensor([index])
        target['area'] = area
        target['iscrowd'] = iscrowd

        if self.transforms:
            sample = {
                'image': image,
                'bboxes': target['boxes'],
                'labels': labels
            }
            sample = self.transforms(**sample)
            image = sample['image']
            
            target['boxes'] = torch.tensor(sample['bboxes'])

        return image, target, image_id

    def __len__(self) -> int:
        return self.image_ids.shape[0]

In [4]:
def get_train_transform():
    return A.Compose([
        A.Flip(0.5),
#         A.RandomCrop(width=100, height=100),
#         A.resize(255,100),
        ToTensorV2(p=1.0)
    ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})

def get_valid_transform():
    return A.Compose([
        ToTensorV2(p=1.0)
    ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})



In [5]:
def collate_fn(batch):
    return tuple(zip(*batch))

train_dataset = Dataset(train_df, './train_data/images', get_train_transform())
valid_dataset = Dataset(val_df, './train_data/images', get_valid_transform())



train_data_loader = DataLoader(
    train_dataset,
    batch_size=2,
    shuffle=False,
    collate_fn=collate_fn
)

valid_data_loader = DataLoader(
    valid_dataset,
    batch_size=1,
    shuffle=False,
    collate_fn=collate_fn
)


In [6]:
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
# replace the classifier with a new one, that has num_classes which is user-defined
num_classes = 3  # mask + without mask + background
# get number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features
# replace the pre-trained head with a new one
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

In [7]:
# device = torch.device('cpu') 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [8]:
torch.cuda.empty_cache()
model.to(device)
params = [p for p in model.parameters() if p.requires_grad]
#optimizer = torch.optim.Adam(params, lr=0.01)
optimizer = torch.optim.SGD(params, lr=0.01,momentum=0.9, weight_decay=0.0005,nesterov=True)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,step_size=3,gamma=0.1, verbose=True)
num_epochs=100

Adjusting learning rate of group 0 to 1.0000e-02.


In [9]:
from engine import train_one_epoch, evaluate
# import utils

for epoch in range(num_epochs):
    # train for one epoch, printing every 10 iterations
    train_one_epoch(model, optimizer, train_data_loader, device, epoch, print_freq=10)
    # update the learning rate
    lr_scheduler.step()
    # evaluate on the test dataset
    evaluate(model, valid_data_loader, device=device)

print("That's it!")

Epoch: [0]  [  0/412]  eta: 0:11:49  lr: 0.000034  loss: 2.6455 (2.6455)  loss_classifier: 1.0720 (1.0720)  loss_box_reg: 0.2102 (0.2102)  loss_objectness: 1.1271 (1.1271)  loss_rpn_box_reg: 0.2363 (0.2363)  time: 1.7217  data: 0.0340  max mem: 3842
Epoch: [0]  [ 10/412]  eta: 0:03:23  lr: 0.000277  loss: 1.7925 (1.7064)  loss_classifier: 0.8299 (0.8101)  loss_box_reg: 0.2245 (0.2640)  loss_objectness: 0.4910 (0.5333)  loss_rpn_box_reg: 0.0635 (0.0990)  time: 0.5060  data: 0.0251  max mem: 3842
Epoch: [0]  [ 20/412]  eta: 0:03:01  lr: 0.000520  loss: 1.4507 (1.4720)  loss_classifier: 0.5371 (0.6367)  loss_box_reg: 0.3409 (0.3462)  loss_objectness: 0.2505 (0.4014)  loss_rpn_box_reg: 0.0611 (0.0878)  time: 0.3988  data: 0.0252  max mem: 4021
Epoch: [0]  [ 30/412]  eta: 0:02:50  lr: 0.000764  loss: 1.0787 (1.3190)  loss_classifier: 0.4240 (0.5594)  loss_box_reg: 0.4332 (0.3803)  loss_objectness: 0.1449 (0.3073)  loss_rpn_box_reg: 0.0190 (0.0720)  time: 0.4129  data: 0.0254  max mem: 4021


Epoch: [0]  [330/412]  eta: 0:00:32  lr: 0.008055  loss: 0.1777 (0.5438)  loss_classifier: 0.0570 (0.2038)  loss_box_reg: 0.0873 (0.2688)  loss_objectness: 0.0030 (0.0533)  loss_rpn_box_reg: 0.0016 (0.0180)  time: 0.3856  data: 0.0192  max mem: 4034
Epoch: [0]  [340/412]  eta: 0:00:28  lr: 0.008299  loss: 0.1426 (0.5324)  loss_classifier: 0.0557 (0.1996)  loss_box_reg: 0.0873 (0.2635)  loss_objectness: 0.0030 (0.0518)  loss_rpn_box_reg: 0.0016 (0.0175)  time: 0.4004  data: 0.0187  max mem: 4034
Epoch: [0]  [350/412]  eta: 0:00:24  lr: 0.008542  loss: 0.1356 (0.5215)  loss_classifier: 0.0510 (0.1955)  loss_box_reg: 0.0843 (0.2585)  loss_objectness: 0.0012 (0.0504)  loss_rpn_box_reg: 0.0020 (0.0171)  time: 0.4065  data: 0.0177  max mem: 4034
Epoch: [0]  [360/412]  eta: 0:00:20  lr: 0.008785  loss: 0.1356 (0.5117)  loss_classifier: 0.0486 (0.1918)  loss_box_reg: 0.0867 (0.2541)  loss_objectness: 0.0013 (0.0491)  loss_rpn_box_reg: 0.0019 (0.0167)  time: 0.4087  data: 0.0183  max mem: 4034


  4%|▎         | 9/249 [00:00<00:02, 84.11it/s]

Epoch: [0]  [411/412]  eta: 0:00:00  lr: 0.010000  loss: 0.0694 (0.4631)  loss_classifier: 0.0242 (0.1742)  loss_box_reg: 0.0395 (0.2304)  loss_objectness: 0.0011 (0.0437)  loss_rpn_box_reg: 0.0007 (0.0148)  time: 0.3735  data: 0.0178  max mem: 4300
Epoch: [0] Total time: 0:02:44 (0.3997 s / it)
Adjusting learning rate of group 0 to 1.0000e-02.


100%|██████████| 249/249 [00:02<00:00, 93.68it/s]


...

Epoch: [99]  [260/412]  eta: 0:01:00  lr: 0.000000  loss: 0.1783 (0.2837)  loss_classifier: 0.0517 (0.0956)  loss_box_reg: 0.1232 (0.1733)  loss_objectness: 0.0006 (0.0067)  loss_rpn_box_reg: 0.0023 (0.0081)  time: 0.3973  data: 0.0216  max mem: 4300
Epoch: [99]  [270/412]  eta: 0:00:56  lr: 0.000000  loss: 0.1077 (0.2790)  loss_classifier: 0.0363 (0.0942)  loss_box_reg: 0.0580 (0.1703)  loss_objectness: 0.0002 (0.0065)  loss_rpn_box_reg: 0.0008 (0.0079)  time: 0.3880  data: 0.0199  max mem: 4300
Epoch: [99]  [280/412]  eta: 0:00:52  lr: 0.000000  loss: 0.1214 (0.2744)  loss_classifier: 0.0364 (0.0929)  loss_box_reg: 0.0783 (0.1675)  loss_objectness: 0.0002 (0.0064)  loss_rpn_box_reg: 0.0013 (0.0077)  time: 0.3824  data: 0.0210  max mem: 4300
Epoch: [99]  [290/412]  eta: 0:00:48  lr: 0.000000  loss: 0.1214 (0.2689)  loss_classifier: 0.0414 (0.0909)  loss_box_reg: 0.0783 (0.1643)  loss_objectness: 0.0003 (0.0062)  loss_rpn_box_reg: 0.0012 (0.0075)  time: 0.3851  data: 0.0212  max mem: 4

  4%|▎         | 9/249 [00:00<00:02, 86.58it/s]

Epoch: [99]  [411/412]  eta: 0:00:00  lr: 0.000000  loss: 0.0328 (0.2098)  loss_classifier: 0.0115 (0.0713)  loss_box_reg: 0.0198 (0.1283)  loss_objectness: 0.0001 (0.0047)  loss_rpn_box_reg: 0.0003 (0.0055)  time: 0.3702  data: 0.0166  max mem: 4300
Epoch: [99] Total time: 0:02:42 (0.3946 s / it)
Adjusting learning rate of group 0 to 1.0000e-35.


100%|██████████| 249/249 [00:02<00:00, 94.37it/s]


creating index...
index created!
Test:  [  0/249]  eta: 0:00:24  model_time: 0.0890 (0.0890)  evaluator_time: 0.0010 (0.0010)  time: 0.1000  data: 0.0100  max mem: 4300
Test:  [100/249]  eta: 0:00:13  model_time: 0.0750 (0.0751)  evaluator_time: 0.0010 (0.0019)  time: 0.0881  data: 0.0101  max mem: 4300
Test:  [200/249]  eta: 0:00:04  model_time: 0.0770 (0.0748)  evaluator_time: 0.0010 (0.0016)  time: 0.0885  data: 0.0102  max mem: 4300
Test:  [248/249]  eta: 0:00:00  model_time: 0.0750 (0.0749)  evaluator_time: 0.0010 (0.0015)  time: 0.0877  data: 0.0107  max mem: 4300
Test: Total time: 0:00:21 (0.0875 s / it)
Averaged stats: model_time: 0.0750 (0.0749)  evaluator_time: 0.0010 (0.0015)
Accumulating evaluation results...
DONE (t=0.07s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.133
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.216
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.

In [10]:
import datetime

now = datetime.datetime.now()
now_str=now.strftime("%Y%m%d")
path = 'model/'+now_str+'fasterRCNN.pth'

torch.save(model,path)